### ERA5 Data Extraction for Census Places

In [ ]:
#%pip install google
#%conda install google
#%pip install --upgrade google-api-python-client
#%pip install --upgrade ee
#%pip install earthengine-api --upgrade

In [1]:
import numpy as np
import re
import pandas as pd
import geopandas as gpd
import gdown
import ee
import google
import os

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


### Get ERA5 hourly data by Census places

In [3]:
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
hourly2022 = era5.filterDate('2022-01-01', '2022-01-02').select('temperature_2m')

In [4]:
# AK Census places as feature collection
ak_places = ee.FeatureCollection('projects/geospatial-data-kpleung/assets/akplaces_2010')

In [ ]:
# reducer 
def reduceByPlaces(image):
  return image.reduceRegions(collection = ak_places,
                             reducer = ee.Reducer.mean().setOutputs(['avg_air_temp']))

placesHourly2022 = hourly2022.map(reduceByPlaces).flatten()

In [ ]:
## Run with caution: expected runtime: 45m

# task = ee.batch.Export.table.toDrive(**{
#   'collection': placesHourly2022,
#   'description': 'Places Average Temp by Hourly 2022',
#   'folder': 'EarthEngine',
#   'selectors': ['system:index','NAME', 'FIPS', 'avg_air_temp'], 
#   'fileFormat': 'CSV'
# })
# task.start()

In [5]:
# download the result from Google drive (later change to Github submodule)
url = "https://drive.google.com/file/d/1T1vRS8OU1S89sfaoDCynN9BxAKRlGRPO/view?usp=drive_link"
output_path = "../Data/"
gdown.download(url, output_path, quiet=False,fuzzy=True)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1T1vRS8OU1S89sfaoDCynN9BxAKRlGRPO
From (redirected): https://drive.google.com/uc?id=1T1vRS8OU1S89sfaoDCynN9BxAKRlGRPO&confirm=t&uuid=7927446a-026c-4a3f-a776-b0f4f8d038e1
To: /Users/brianleung/Documents/Python-Projects/DSSG2023-Heating-Loads/Data/Places Average Temp by Hourly 2022.csv
100%|██████████| 226M/226M [00:10<00:00, 21.5MB/s] 


'../Data/Places Average Temp by Hourly 2022.csv'

In [ ]:
# replicating above with the full hourly data
temp_data = pd.read_csv("../Data/Places Average Temp by Hourly 2022.csv")
temp_data.head()

In [ ]:
temp_data['Month'] = temp_data['system:index'].str[4:6]
temp_data['db_temp'] = (1.8 * (temp_data['avg_air_temp']-273)) + 32
temp_data['timestamp'] = temp_data['system:index'].str[:11].str.replace("T","")
temp_data['timestamp'] = pd.to_datetime(temp_data['timestamp'], format ='%Y%m%d%H')
data = temp_data[['FIPS','NAME','timestamp', 'db_temp', 'Month']]
data.tail(10)

In [ ]:
df = data
df_dict = {str(g): d for g, d in df.groupby('FIPS')}

In [ ]:
for i in df_dict:
    df_dict[i].to_csv ('../Data/ERA5/'+ i + '.csv', index = None, header=True) 

In [ ]:
for i in df_dict:
    df_dict[i].to_pickle ('../Data/ERA5/'+ i + '.pkl') 